**!** *Tip: type `shift+enter` OR click the ` >| Run ` button above to exectue a cell*

### Init

In [1]:
import sys
sys.path.append("./src")

from models import *
from processing import *

### Setup (change initialization cell)

#### OLIM Coil Parameters

In [2]:
coil = Coil(wired_in=Coil.SERIES, current=Coil.DC, material=Coil.COPPER, wire_geometry=Coil.CIRCLE)
tube = Tube(ID=18, OD=21, effective_length=150, unit="mm")

In [3]:
p = Process(coil, tube)

### Exploring Parameters

#### wire diameter and coil height

Explore against one or multiple parameters (ctrl/Command+click, or shift+click)

In [4]:
p.explore_wire_diameter(resolution=0.001, debug=False)

GridBox(children=(GridBox(children=(Select(disabled=True, index=3, layout=Layout(grid_area='resistance', heigh…

After exploring, assign the parameters:

In [ ]:
coil.set_wire_d(1.0, unit="mm") 
coil.set_layer_count(14)  # comment if you wish to explore this value
coil.mag_Br = 1.45  #  N45 neodynium magnet

Once we have explored values for wire diameter and current, 
we can explore force against extension of plunger and layer height

### Explore Actuator Force

In [ ]:
p.explore_actuator_force()

# Source of equations https://core.ac.uk/download/pdf/9554327.pdf page 14
# https://danielteodesigntechnology.wordpress.com/theoretical/supplement-lorentz-force-actuation/

### Playground

In [ ]:
coil.set_wire_d(1.0, unit='mm')

coil.set_layer_count(14)
# OR
#  coil.set_layer_height(14.0, unit='mm')
#  this will automatically calculate layer count

Electrical Properties

In [ ]:
I, R = coil.current, coil.resistance; V = I * R
p.out(u"Max Parameters:\n"
       "V=%0.8fV \n"
       "I=%0.2fA \n"
       "R=%0.10f\u2126 \n"
       "P=%0.2fW" % (V, I, R, V*I))

Physical Properties

In [ ]:
wire_d, coil_m, coil_L, coil_h, N_turns = coil.get_wire_d(unit='mm'), \
                                          coil.get_mass(unit='kg'), \
                                          coil.get_length(unit='m'), \
                                          coil.get_height(unit="mm"), \
                                          coil.get_number_of_turns()
p.out("wire diameter=%.2fmm \n"
      "coil mass=%.2fkg \n"
      "coil length=%.1fm \n"
      "coil OD=%.2fmm \n"
      "number of turns=%.0f"% (wire_d, coil_m, coil_L, coil_h*2 + tube.OD*1000, N_turns))

Force of actuator, represented as the vector (magnitude, direction) in the axis of travel.

In [ ]:
samples = 11  # how many data points over the stroke of the actuator
for z in [(x/(samples-1))*tube.effective_length for x in range(0, samples, 1)]:
    print("x=%.3fm," % z, end=' F=')
#     Bmag = 1.241  #  N38 Neodydlium magnet
    Bmag = 1.35 #  N42 Neodydlium magnet
    Bsol_z = coil.calculate_magnetic_field(_coil_layers=coil.layer_count, z=z)

    d = tube.ID
    A = math.pi*pow(d/2 + (tube.OD - tube.ID)/2 + 0.001, 2)
    mu = 4.0 * math.pi * pow(10, -7)

    f_z = (Bsol_z[0] - abs(Bsol_z[1])) * Bmag * A / mu
    print("%.3f" % f_z, end=' ')  # resolved
    print("(%.3f, %.3f)" % (Bsol_z[0] * Bmag * A / mu, Bsol_z[1] * Bmag * A / mu))  # components

In [ ]:
F = coil.number_of_turns * 3.1415 * coil.avg_D * 0.25 * 2
p.out("force=%.2fN \n"
      "number of turns=%.0f \n"
      "coil avg D=%.1fmm \n" % (F, coil.number_of_turns, coil.avg_D*1000))

In [ ]:
coil.calculate_magnetic_field??

In [ ]:
    coil.calculate_force_between_plunger_and_coil??